In [11]:
#
from util.PrepareData import ACCESS_BARRA_crps 
import util.data_processing_tool as dpt

from datetime import timedelta, date, datetime
import os
import sys
import numpy as np

class aaa(object):
    def __init__(self,lead):
        self.ensemble_access=['e01','e02','e03','e04','e05','e06','e07','e08','e09','e10','e11']
        self.lead_time=lead
        self.files=self.get_filename_with_time_order()
    def get_filename_with_time_order(self):
        _files = []
        for mm in range(1,13):
            for dd in [1,9,17,25]:
#                 for i in range(self.lead_time,self.lead_time+1):
#                 for en in ['e01','e02','e03','e04','e05','e06','e07','e08','e09','e10','e11']:
                path=[]
                date_time=date(2012, mm, dd)
                barra_date=date_time+timedelta(self.lead_time)
                path.append(date_time)
                path.append(barra_date)
                path.append(self.lead_time)
                _files.append(path)
        return _files
    def __getitem__(self,idx):
        return self.files[idx]

def load_climatology_data(l):
    data=aaa(l)
    climtology_lead_time=[]
    climatology_data=np.load('./save/crps/climatology_all_lead_time.npy')
    dates_needs=date_range(date(2012, 1, 1),date(2013, 7, 29))
    date_map=np.array(dates_needs)
    for _,target_date,_ in data.files:
        idx=np.where(date_map==target_date)[0]
        climtology_lead_time.append(climatology_data[idx][0])
    return np.array(climtology_lead_time)

def date_range(start_date, end_date):
    """This function takes a start date and an end date as datetime date objects.
    It returns a list of dates for each date in order starting at the first date and ending with the last date"""
    return [start_date + timedelta(x) for x in range((end_date - start_date).days + 1)]



In [ ]:
# draw confidnece fiture
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
land=np.load("./save/crps/whole_calibration/lead_time"+str(0)+"_whole.npy").mean(0)
x=list(range(1,31))

plt.figure(dpi=100,figsize=(15,5))

data=[]
mean_my=[]
# for i in range(30):
#     a=np.load("./save_vdsr_pr_best_test/lead_time"+str(i)+"_50station_my.npy")
#     mean_my.append(1-a.mean()/station50_int[i])
# for q in [95,75,50,25,5]:
#     data[q]=[]
for q in [90,75,50,25,10]:
    t=[]
    for lead_time in range(30):
        
        my=np.load("./save/crps/my_217/lead_time"+str(lead_time)+".npy").mean(0)[~np.isinf(land)]
        climat=load_climatology_data(lead_time).mean(0)[~np.isinf(land)]
#         print(my)
#         t.append(1-(np.percentile(my,q)/climat))
        t.append(np.percentile(    1-my/climat,q)   )

    data.append(t)

data_np=np.array(data)
# plt.plot(x,mean_my,label="",color="b")#mean
plt.plot(x,data_np[2,:],label="",color="r")#median
    
plt.plot(x,[0]*30,color="#000000")
plt.fill_between(x,data_np[0,:],data_np[1,:],color="#cacaca")
plt.fill_between(x,data_np[1,:],data_np[2,:],color="#989898")
plt.fill_between(x,data_np[2,:],data_np[3,:],color="#989898")
plt.fill_between(x,data_np[3,:],data_np[4,:],color="#cacaca")
# plt.xlim(0,30)
# plt.grid()
plt.xlabel(" Leadtime (day)")
plt.ylabel(" SS_CRPS")
plt.ylim(-1.2,1.2)

pdf = PdfPages('ours_BI.pdf')
pdf.savefig()
pdf.close() 

In [33]:
list(station_dict.keys())

['CARNAMAH',
 'MULLEWA',
 'WONGAN HILLS',
 'BADGINGARRA RESEARCH STN',
 'MOUNT BARKER',
 'ESPERANCE',
 'BENCUBBIN',
 'KELLERBERRIN',
 'BEVERLEY',
 'CORRIGIN',
 'HYDEN',
 'NARROGIN',
 'ONGERUP',
 'RAVENSTHORPE',
 'SALMON GUMS RES.STN.',
 'CEDUNA AMO',
 'CLEVE',
 'KYANCUTTA',
 'STREAKY BAY',
 'YONGALA',
 'PRICE',
 'WAROOKA',
 'ROSEDALE (TURRETFIELD RESEARCH CENTRE)',
 'MENINGIE',
 'KEITH',
 'SPRINGSURE COMET ST',
 'OAKEY AERO',
 'SURAT',
 'LAKE VICTORIA STORAGE',
 'COLLARENEBRI (ALBERT ST)',
 'BALRANALD (RSL)',
 'PEAK HILL POST OFFICE',
 'CONDOBOLIN AG RESEARCH STN',
 'NYNGAN AIRPORT',
 'TRANGIE RESEARCH STATION AWS',
 'MUNGINDI POST OFFICE',
 'QUIRINDI POST OFFICE',
 'ORANGE AGRICULTURAL INSTITUTE',
 'WAGGA WAGGA AMO',
 'GRENFELL (MANGANESE RD)',
 'COROWA AIRPORT',
 'NARRANDERA AIRPORT AWS',
 'HILLSTON AIRPORT',
 'LAKE CARGELLIGO AIRPORT',
 'MILDURA AIRPORT',
 'OUYEN (POST OFFICE)',
 'WARRACKNABEAL MUSEUM',
 'ECHUCA AERODROME',
 'KERANG',
 'ARARAT PRISON']

In [ ]:
#save 50 station file to csv
#in style 1

if option=='50':
    from util.constant_param import station_50_index_for_size_of_hr_sr as station_dict
if option=='214':
    from util.constant_param import station_214_index_for_size_of_hr_sr as station_dict

with open(file_dir, "w", newline='') as file:
    csv_file = csv.writer(file)
    for i in station_dict.keys():
        idx=station_dict[i]
        head = ["leading","climatology_11",'bilinear interpolation',"our model","calibration",'cspr_ss_mean_bilinearinterpolation_against_climatology',"cspr_ss_mean_ourmodel_against_climatology","cspr_ss_mean_calibration_against_climatology"]

    
    

#         t=np.load("./save/crps/whole_calibration/lead_time"+str(0)+"_whole.npy").mean(0)
#         station_index=np.zeros((t.shape))
#         for i in station_dict.keys():
#             station_index[station_dict[i]]=1
#         with open(file_dir, "w", newline='') as file:
#             csv_file = csv.writer(file)
#             head = ["leading","climatology_11",'bilinear interpolation',"our model","calibration",'cspr_ss_mean_bilinearinterpolation_against_climatology',"cspr_ss_mean_ourmodel_against_climatology","cspr_ss_mean_calibration_against_climatology"]
#             csv_file.writerow(head)
#             t=np.load("./save/crps/whole_calibration/lead_time"+str(0)+"_whole.npy").mean(0)
#             for lead_time in range(217):
#                 line=[lead_time,]

#                 cali=np.load("./save/crps/whole_calibration/lead_time"+str(lead_time)+"_whole.npy").mean(0)[station_index==1]
#                 climat=load_climatology_data(lead_time).mean(0)[station_index==1]
#                 my=np.load("./save/crps/my_217/lead_time"+str(lead_time)+".npy").mean(0)[station_index==1]

#                 BI=np.load("./save/crps/bi_217/lead_time"+str(lead_time)+".npy").mean(0)[station_index==1]


#                 my_mean=my.mean()
#                 cali_mean=cali.mean()
#                 qm_mean=climat.mean()
#                 BI_mean=BI.mean()

#         #         my_cali=(1- my/cali)
#                 my_qm=(1- my/climat)
#                 cali_qm=(1- cali/climat)
#                 bi_qm=(1- BI/climat)
#         #         np.save("./save/crps/result/my_cali"+str(lead_time),my_cali)
#         #         np.save("./save/crps/result/my_qm"+str(lead_time),my_qm)
#         #         np.save("./save/crps/result/cali_qm"+str(lead_time),cali_qm)
#         #         np.save("./save/crps/result/bi_qm"+str(lead_time),bi_qm)
#         #         cspr_ss_mean_my_cali=my_cali.mean()
#                 cspr_ss_mean_my_qm=my_qm.mean()
#                 cspr_ss_mean_cali_qm=cali_qm.mean()
#                 cspr_ss_mean_bi_qm=bi_qm.mean()

#                 line.append(qm_mean)
#                 line.append(BI_mean)
#                 line.append(my_mean)
#                 line.append(cali_mean)
#                 line.append(cspr_ss_mean_bi_qm)
#                 line.append(cspr_ss_mean_my_qm)
#                 line.append(cspr_ss_mean_cali_qm)
#         #         line.append(cspr_ss_mean_my_cali)
#                 csv_file.writerow(line)

In [ ]:
#calculate csv file
import csv
import numpy as np


def calculate_csv_file(file_dir="./save/mean_table_mean_crps_mean_ss.csv",option='whole'):
    '''
    @option:'whole', '50', '214'
            'whole' presents the whole australia.
            '50': means 50 observation station.
            '214' presents 214 new observation application station.
    '''
    if option is "whole":
        with open(file_dir, "w", newline='') as file:
            csv_file = csv.writer(file)
            head = ["leading","climatology_11",'bilinear interpolation',"our model","calibration",'cspr_ss_mean_bilinearinterpolation_against_climatology',"cspr_ss_mean_ourmodel_against_climatology","cspr_ss_mean_calibration_against_climatology"]
            csv_file.writerow(head)
            t=np.load("./save/crps/whole_calibration/lead_time"+str(0)+"_whole.npy").mean(0)
            for lead_time in range(217):
                line=[lead_time,]
                cali=np.load("./save/crps/whole_calibration/lead_time"+str(lead_time)+"_whole.npy").mean(0)[~np.isinf(t)]

                climat=load_climatology_data(lead_time).mean(0)[~np.isinf(t)]
                my=np.load("./save/crps/my_217/lead_time"+str(lead_time)+".npy").mean(0)[~np.isinf(t)]

                BI=np.load("./save/crps/bi_217/lead_time"+str(lead_time)+".npy").mean(0)[~np.isinf(t)]


                my_mean=my.mean()

        #         print(my_mean.shape)
                cali_mean=cali.mean()
                qm_mean=climat.mean()
                BI_mean=BI.mean()

        #         my_cali=(1- my/cali)
                my_qm=(1- my/climat)
                cali_qm=(1- cali/climat)
                bi_qm=(1- BI/climat)
        #         np.save("./save/crps/result/my_cali"+str(lead_time),my_cali)
        #         np.save("./save/crps/result/my_qm"+str(lead_time),my_qm)
        #         np.save("./save/crps/result/cali_qm"+str(lead_time),cali_qm)
        #         np.save("./save/crps/result/bi_qm"+str(lead_time),bi_qm)

        #         cspr_ss_mean_my_cali=my_cali.mean()
                cspr_ss_mean_my_qm=my_qm.mean()
                cspr_ss_mean_cali_qm=cali_qm.mean()
                cspr_ss_mean_bi_qm=bi_qm.mean()

                line.append(qm_mean)
                line.append(BI_mean)
                line.append(my_mean)
                line.append(cali_mean)
                line.append(cspr_ss_mean_bi_qm)
                line.append(cspr_ss_mean_my_qm)
                line.append(cspr_ss_mean_cali_qm)
        #         line.append(cspr_ss_mean_my_cali)
                csv_file.writerow(line)
    else:
        if option=='50':
            from util.constant_param import station_50_index_for_size_of_hr_sr as station_dict
        if option=='214':
            from util.constant_param import station_214_index_for_size_of_hr_sr as station_dict

        t=np.load("./save/crps/whole_calibration/lead_time"+str(0)+"_whole.npy").mean(0)
        station_index=np.zeros((t.shape))
        for i in station_dict.keys():
            station_index[station_dict[i]]=1
        with open(file_dir, "w", newline='') as file:
            csv_file = csv.writer(file)
            head = ["leading","climatology_11",'bilinear interpolation',"our model","calibration",'cspr_ss_mean_bilinearinterpolation_against_climatology',"cspr_ss_mean_ourmodel_against_climatology","cspr_ss_mean_calibration_against_climatology"]
            csv_file.writerow(head)
            t=np.load("./save/crps/whole_calibration/lead_time"+str(0)+"_whole.npy").mean(0)
            for lead_time in range(217):
                line=[lead_time,]

                cali=np.load("./save/crps/whole_calibration/lead_time"+str(lead_time)+"_whole.npy").mean(0)[station_index==1]
                climat=load_climatology_data(lead_time).mean(0)[station_index==1]
                my=np.load("./save/crps/my_217/lead_time"+str(lead_time)+".npy").mean(0)[station_index==1]

                BI=np.load("./save/crps/bi_217/lead_time"+str(lead_time)+".npy").mean(0)[station_index==1]


                my_mean=my.mean()
                cali_mean=cali.mean()
                qm_mean=climat.mean()
                BI_mean=BI.mean()

        #         my_cali=(1- my/cali)
                my_qm=(1- my/climat)
                cali_qm=(1- cali/climat)
                bi_qm=(1- BI/climat)
        #         np.save("./save/crps/result/my_cali"+str(lead_time),my_cali)
        #         np.save("./save/crps/result/my_qm"+str(lead_time),my_qm)
        #         np.save("./save/crps/result/cali_qm"+str(lead_time),cali_qm)
        #         np.save("./save/crps/result/bi_qm"+str(lead_time),bi_qm)
        #         cspr_ss_mean_my_cali=my_cali.mean()
                cspr_ss_mean_my_qm=my_qm.mean()
                cspr_ss_mean_cali_qm=cali_qm.mean()
                cspr_ss_mean_bi_qm=bi_qm.mean()

                line.append(qm_mean)
                line.append(BI_mean)
                line.append(my_mean)
                line.append(cali_mean)
                line.append(cspr_ss_mean_bi_qm)
                line.append(cspr_ss_mean_my_qm)
                line.append(cspr_ss_mean_cali_qm)
        #         line.append(cspr_ss_mean_my_cali)
                csv_file.writerow(line)


In [27]:
option='50'
if option=='50':
    from util.constant_param import station_50_index_for_size_of_hr_sr as station_dict
if option=='214':
    from util.constant_param import station_214_index_for_size_of_hr_sr as station_dict
t=np.load("./save/crps/whole_calibration/lead_time"+str(0)+"_whole.npy").mean(0)
station_index=np.zeros((t.shape))
for i in station_dict.keys():
    station_index[station_dict[i]]=1
with open("./save/50_station_mean_table_mean_crps_ss.csv", "w", newline='') as file:
    csv_file = csv.writer(file)
    head = ["leading","climatology_11",'bilinear interpolation',"our model","calibration",'cspr_ss_mean_bilinearinterpolation_against_climatology',"cspr_ss_mean_ourmodel_against_climatology","cspr_ss_mean_calibration_against_climatology"]
    csv_file.writerow(head)
    t=np.load("./save/crps/whole_calibration/lead_time"+str(0)+"_whole.npy").mean(0)
    for lead_time in range(217):
        line=[lead_time,]

        cali=np.load("./save/crps/whole_calibration/lead_time"+str(lead_time)+"_whole.npy").mean(0)[station_index==1]
        climat=load_climatology_data(lead_time).mean(0)[station_index==1]
        my=np.load("./save/crps/my_217/lead_time"+str(lead_time)+".npy").mean(0)[station_index==1]

        BI=np.load("./save/crps/bi_217/lead_time"+str(lead_time)+".npy").mean(0)[station_index==1]


        my_mean=my.mean()
        cali_mean=cali.mean()
        qm_mean=climat.mean()
        BI_mean=BI.mean()

#         my_cali=(1- my/cali)
        my_qm=(1- my/climat)
        cali_qm=(1- cali/climat)
        bi_qm=(1- BI/climat)
#         np.save("./save/crps/result/my_cali"+str(lead_time),my_cali)
#         np.save("./save/crps/result/my_qm"+str(lead_time),my_qm)
#         np.save("./save/crps/result/cali_qm"+str(lead_time),cali_qm)
#         np.save("./save/crps/result/bi_qm"+str(lead_time),bi_qm)
#         cspr_ss_mean_my_cali=my_cali.mean()
        cspr_ss_mean_my_qm=my_qm.mean()
        cspr_ss_mean_cali_qm=cali_qm.mean()
        cspr_ss_mean_bi_qm=bi_qm.mean()

        line.append(qm_mean)
        line.append(BI_mean)
        line.append(my_mean)
        line.append(cali_mean)
        line.append(cspr_ss_mean_bi_qm)
        line.append(cspr_ss_mean_my_qm)
        line.append(cspr_ss_mean_cali_qm)
#         line.append(cspr_ss_mean_my_cali)
        csv_file.writerow(line)


array([0.75082933, 0.39796467, 0.67596422, 0.88570552, 0.99554862,
       0.46132181, 1.07482679, 1.28955316, 0.64767965, 0.43702554,
       0.92396885, 0.84009873, 0.83099781, 0.65505628, 0.51406468,
       0.5864177 , 0.39942535, 0.64169532, 1.68851628, 0.5474031 ,
       0.70903717, 0.55017177, 0.80886314, 2.96433262, 1.05876775,
       0.4467125 , 0.8179214 , 1.00748534, 0.6512305 , 0.66535109,
       0.48018446, 1.72451818, 0.48924617, 0.50384647, 0.3545763 ,
       0.5643073 , 4.07559364, 0.39359319, 3.8407271 , 2.09929895,
       0.47683023, 0.5394267 , 1.05181824, 1.18882912, 3.90639974,
       1.47378273, 0.95371375, 1.90446604, 1.22869173, 1.65107286])

SyntaxError: invalid decimal literal (<ipython-input-1-07e05b34a482>, line 1)